In [ ]:
import re
import sqlite3
import datetime
import glob

# Create the Database 

In [ ]:
#create the local database
db = sqlite3.connect("northstar_athletics.db")

In [ ]:
#set the cursor
cur = db.cursor()

In [ ]:
cur

In [ ]:
#Create the Table

#DROP TABLE IF EXISTS is used to avoid an error if we try to drop the table and it doesn't exist
cur.execute('''DROP TABLE IF EXISTS transactions''')
#create table "transactions" and define columns and datatypes
cur.execute('''CREATE TABLE transactions (
    date TEXT, 
    ticket_num INT,
    emp_num INT,
    trans_type TEXT,
    cust TEXT, 
    sku TEXT, 
    item TEXT, 
    size TEXT, 
    quan_sold INT, 
    quan_returned INT, 
    retail_price FLOAT, 
    price FLOAT,
    unmatched FLOAT, 
    perks FLOAT, 
    markdown FLOAT, 
    returned FLOAT, 
    net_sale FLOAT, 
    cost FLOAT, 
    gp_percent INT,
    payment_type TEXT,
    time TEXT
    )''')

db.commit()

# Write Transaction Rows to Database

## Set the path to the Sales Journal files

In [ ]:
#Set the path to the folder containing all the sales journal files 
path = "Sales Journal Clean/*.txt"

#Check to make sure it's reading all the file names
for fname in glob.glob(path) :
    print(fname)

## Define Regular Expressions

In [ ]:
start_of_new_ticket = re.compile(r'D[ ]{1,}[0-9]{3,5}')
good_row_re = re.compile(r'D[ ]{7}[0-9]')
date_re = re.compile(r'1?[0-9]/[0-9]{1,2}/[0-9]{4}')
time_re = re.compile(r'[0-9]{2}:[0-9]{2} [AP][M]')


## Loop to Write Transaction Rows to Database

In [ ]:
for fname in glob.glob(path) :    
    
    hit_start = False
    new_ticket = False
    have_date = False
    is_gift = False
    
    ## is_return 

    with open(fname, 'r') as ifile: 
        for row in ifile :

            if not have_date and date_re.search(row) :
                the_date = date_re.search(row).group(0)
                the_date = datetime.datetime.strptime(the_date, "%m/%d/%Y").strftime("%Y-%m-%d")
                have_date = True

            if hit_start :
                if start_of_new_ticket.search(row) and not new_ticket and "Regular" in row:
                    if "Ticket VOIDED" and "*** No Sale ***" not in row:
                        new_ticket = True

                        output_rows = []
                        
                        ticket_num = row[1:7].strip()
                        emp_num = row[8:10]
                        trans_type = "Regular"
                        cust = row[25:100].strip()
                        
                elif start_of_new_ticket.search(row) and not new_ticket and "Gift Cert" in row:
                    if "Ticket VOIDED" and "*** No Sale ***" not in row:
                        new_ticket = True
                            
                        output_rows = []

                        ticket_num = row[1:7].strip()
                        emp_num = row[8:10]
                        trans_type = "Gift Cert"
                        cust = row[25:100].strip()
                         
                        is_gift = True
                     

                elif new_ticket :
                      # Get payment type
                    if "Totals" in row:
                        payment_type = row[196:211].strip()
                        
                        # is ticket over?    
                    if  row.startswith("NP") or "-"*47 in row :
                        new_ticket = False

                        # Get time
                        if "-"*47 in row : 
                            the_time = time_re.search(row).group(0) 
                        else : 
                            the_time = ""

                        # add rows to the database
                        for o_row in output_rows:
                            o_row.append(payment_type)
                            o_row.append(the_time)
                            
                            
                            cur.execute('''
                            INSERT INTO transactions (date,ticket_num,emp_num,trans_type,cust,sku,item,size,quan_sold,quan_returned,
                                      retail_price,price,unmatched,perks,markdown,returned,net_sale,cost,gp_percent,payment_type,time)
                            VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''', o_row)
                           
                    else :
                        # if not over, process row
                        if good_row_re.search(row) and "Ticket VOIDED" and "*** No Sale ***" not in row:
                            if "Totals" not in row and not is_gift:
                                sku = row[13:29].strip()
                                item = row[29:50].strip()
                                size = row[50:62].strip()
                                quan_sold = row[62:67].strip()
                                quan_returned = row[67:70].strip()
                                retail_price = row[70:79].strip()
                                price = row[79:89].strip()
                                unmatched = row[89:98].strip()
                                perks = row[98:109].strip()
                                markdown = row[109:119].strip()
                                returned = row[119:130].strip()
                                net_sale = row[130:141].strip()
                                cost = row[141:149].strip()
                                gp_percent = row[149:155].strip()
                                payment_type = row[196:211].strip()
                                
                            elif "Totals" not in row and is_gift:
                                sku = row[13:29].strip()
                                item = row[29:50].strip()
                                size = row[50:62].strip()
                                quan_sold = row[62:67].strip()
                                quan_returned = row[67:70].strip()
                                # if is_gift use totals 
                                retail_price = row[186:196].strip()
                                # if is_gift use totals 
                                price = row[186:196].strip()
                                unmatched = row[89:98].strip()
                                perks = row[98:109].strip()
                                markdown = row[109:119].strip()
                                returned = row[119:130].strip()
                                # if is_gift use totals 
                                net_sale = row[186:196].strip()
                                cost = row[141:149].strip()
                                gp_percent = row[149:155].strip()
                                payment_type = row[196:211].strip()
                    
                                
                            output_rows.append([the_date,ticket_num,emp_num,trans_type,cust,sku,item,size,quan_sold,quan_returned,
                                          retail_price,price,unmatched,perks,markdown,returned,net_sale,cost,gp_percent])
                            
                            is_gift = False

            if "-------- SKU --------" in row :
                hit_start = True

db.commit()

In [ ]:
db.commit()